# Schema: XML Schema

Ähnlich wie die Document Type Definition (DTD), ermöglicht auch XML Schema die Spezifikation von XML Dokumente und somit das Validieren solcher Dokumente. Ziel beider Grammatiken ist die Interoperabilität zwischen entwickelten Systeme zu ermöglichen oder erhöhen. In dieser Übung schauen wir uns XML Schema etwas genauer in der Praxis an. Führen Sie zuerst den folgenden Codeblock aus und machen Sie dann der Reihe nach weiter. Beantworten Sie die Fragen (falls zutreffend). Zum Schluss, schreiben Sie ein eigenes XML Schema und ein exemplarisches XML Dokument dafür. Stellen Sie sicher, dass das XML Dokument wohlgeformt und gültig ist.

In [1]:
import io
from lxml import etree as et

def isvalid(xsd, doc):
    print(et.XMLSchema(et.parse(io.StringIO(xsd))).validate(et.fromstring(doc)))

In [4]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="discography"/>
</xs:schema>
"""

isvalid(xsd, '<discography/>')
isvalid(xsd, '<albums/>') # Warum ist dieses Dokument nicht gültig? Antwort: Weil "albums" nicht vorher im xsd definiert wurde

True
False


In [7]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="title" type="xs:string"/>
</xs:schema>
"""

isvalid(xsd, '<title>The Dark Side of the Moon</title>')
isvalid(xsd, '<title>1973</title>') # Warum ist dieses Dokument gültig? Antwort: Da "title" als type string definiert wurde, so 
                                    # "1973" als String behandelt

True
True


In [8]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="year" type="xs:decimal"/>
</xs:schema>
"""

isvalid(xsd, '<year>1973</year>')
isvalid(xsd, '<year>The Dark Side of the Moon</year>') # Warum ist dieses Dokument nicht gültig? Antwort: "year" wurde als
                                                       # "decimal" definiert, also als Zahl. Hier ist es aber keine Zahl

True
False


In [ ]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="year" type="xs:gYear"/>
</xs:schema>
"""

isvalid(xsd, '<year>1973</year>')

In [11]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="year" type="xs:gMonth"/>
</xs:schema>
"""

isvalid(xsd, '<year>1973</year>') # Warum ist dieses Dokument nicht gültig? Antwort: In der xsd wurde zuvor definiert das das
                                   # Element "year" als type "Month" besitzt, was hier aber als "Year" geschrieben wurde

False


## Komplexe Elemente Deklarieren: Variante I

In [12]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

True


In [13]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="label" type="xs:string"/>
        <xs:element name="title" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Die Elemente "tile" und "label" sind in der falschen 
                  # Reihenfolge aufgezählt als im xsd zuvor definiert (xs:sequence gibt an in welcher Reihenfolge Elemente
                  # vorkommen sollen)

False


In [16]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<label>Harvest, EMI</label>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: Die Elemente "label" und "title" wurden in der falschen
                  # Reihenfolge als im xsd zuvor definiert angegeben

False


## Komplexe Elemente Deklarieren: Variante II

In [ ]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album" type="AlbumType"/>
  
  <xs:complexType name="AlbumType">
    <xs:sequence>
      <xs:element name="title" type="xs:string"/>
      <xs:element name="label" type="xs:string"/>
    </xs:sequence>
  </xs:complexType>

</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

In [19]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="work" type="WorkType"/>
  <xs:element name="album" type="AlbumType"/>
  <xs:complexType name="WorkType">
    <xs:sequence>
      <xs:element name="title" type="xs:string"/>
    </xs:sequence>
  </xs:complexType>
  <xs:complexType name="AlbumType">
    <xs:complexContent>
      <xs:extension base="WorkType">
        <xs:sequence>
          <xs:element name="label" type="xs:string"/>
        </xs:sequence>
      </xs:extension>
    </xs:complexContent>
 </xs:complexType>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<!-- The order matters -->
<label>Harvest, EMI</label>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: "label" und "title" müssen in der Reihenfolge angegeben werden
                  # wie im xsd definiert wurde

True
True


# Attribute Deklarieren

In [ ]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:attribute name="title" type="xs:string"/>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album title="The Dark Side of the Moon"/>
"""

isvalid(xsd, doc)

In [ ]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:simpleContent>
        <xs:extension base="xs:string">
          <xs:attribute name="year" type="xs:gYear" />
        </xs:extension>
      </xs:simpleContent>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album year="1973">The Dark Side of the Moon</album>
"""

isvalid(xsd, doc)

# Weitere Eigenschaften

In [20]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string" maxOccurs="2"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<title>The Bright Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument gültig? Antwort: In der xsd wurde definiert dass, das Element "title" maximal
                  # 2 mal vorkommen darf, was hier der Fall ist

True


In [21]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:all>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:all>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<label>Harvest, EMI</label>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument gültig? Antwort: In der xsd wurde angegeben das die elemente "label" und "title"
                  # in einer beliebigen Reihenfolge vorkommen dürfen, müssen aber alle einmal vorkommen

True
True


In [ ]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="album">
    <xs:complexType>
      <xs:choice>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:choice>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album>
<title>The Dark Side of the Moon</title>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

doc = """
<album>
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: In der xsd wurde definiert das nur ein Element aus der Liste
                  # "title" und "label" gewählt werden darf, hier wurden aber beide gewählt

In [ ]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="released">
    <xs:simpleType>
      <xs:restriction base="xs:integer">
        <xs:minInclusive value="1950"/>
        <xs:maxInclusive value="2018"/>
      </xs:restriction>
    </xs:simpleType>
  </xs:element>
</xs:schema>
"""

doc = """
<released>1973</released>
"""

isvalid(xsd, doc)

doc = """
<released>2020</released>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: In der xsd wurde zuvor definiert dass,Das Element "released" 
                  # darf nur den Wert 1950 oder 2018 annehmen darf.

In [22]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="author">
    <xs:simpleType>
      <xs:restriction base="xs:string">
        <xs:enumeration value="Roger Waters"/>
        <xs:enumeration value="David Gilmour"/>
        <xs:enumeration value="Syd Barrett"/>
      </xs:restriction>
    </xs:simpleType>
  </xs:element>
</xs:schema>
"""

doc = """
<author>Roger Waters</author>
"""

isvalid(xsd, doc)

doc = """
<author>Jimmy Page</author>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: In der xsd wurde zuvor definiert das der Inhalt des ELementes
                  # "author" nur bestimmte Werte haben darf. "Jimmy Page" ist nicht dabei

True
False


In [23]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="author">
    <xs:simpleType>
      <xs:restriction base="xs:string">
        <xs:pattern value="[A-Z][a-z]+"/>
      </xs:restriction>
    </xs:simpleType>
  </xs:element>
</xs:schema>
"""

doc = """
<author>Waters</author>
"""

isvalid(xsd, doc)

doc = """
<author>gilmour</author>
"""

isvalid(xsd, doc) # Warum ist dieses Dokument nicht gültig? Antwort: In der xsd wurde zuvor definiert dass, der Inhalt des Elementes
                  # "author" mit einem Großbuchstaben anfangen muss, was hier nicht der Fall ist.

True
False


In [ ]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema" 
  targetNamespace="http://example.org"
  elementFormDefault="qualified">
  <xs:element name="album">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="title" type="xs:string"/>
        <xs:element name="label" type="xs:string"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<album xmlns="http://example.org">
<title>The Dark Side of the Moon</title>
<label>Harvest, EMI</label>
</album>
"""

isvalid(xsd, doc)

Denken Sie sich nun ein eigenes XML Dokument aus und erstellen Sie dafür ein XML Schema. Überprüfen Sie das XML Dokument auf Gültigkeit.

In [52]:
xsd = """
<xs:schema xmlns:xs="http://www.w3.org/2001/XMLSchema">
  <xs:element name="persons">
    <xs:complexType>
      <xs:sequence>
        <xs:element name="name">
            <xs:simpleType>
                <xs:restriction base="xs:string">
                    <xs:pattern value="\s[A-Z][a-z]+\s"/>
                </xs:restriction>
            </xs:simpleType>
        </xs:element>
        <xs:element name="surname" type="xs:string"/>
        <xs:element name="birthday" type="xs:decimal" maxOccurs="unbounded"/>
      </xs:sequence>
    </xs:complexType>
  </xs:element>
</xs:schema>
"""

doc = """
<persons>
    <name> Anna </name>
    <surname> Lisa </surname>
    <birthday> 1997 </birthday>
</persons>
"""

isvalid(xsd, doc)

True
